In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Libraries and Imports

In [2]:
%%capture
!pip install -q llama-index llama-index-llms-groq llama-index-embeddings-fastembed llama-index-vector-stores-kdbai llama-index-callbacks-opik llama-index-postprocessor-colbert-rerank llama-index-llms-gemini llama-index
!pip install -q groq kdbai-client opik litellm
!pip install -q pandas torch transformers
!pip install -q tqdm nest_asyncio
!pip install -q qdrant-client llama-index-vector-stores-qdrant
!pip install -q pip install llama-index-retrievers-bm25

In [3]:
from llama_index.core import (
    VectorStoreIndex,
    SummaryIndex,
    StorageContext,
    Settings,
    Document,
    PromptTemplate,
    set_global_handler)
from llama_index.core.node_parser import (
    SentenceSplitter,
    MarkdownNodeParser)
from llama_index.core.llms import ChatMessage
from llama_index.llms.groq import Groq
from llama_index.llms.gemini import Gemini
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.vector_stores.kdbai import KDBAIVectorStore
from llama_index.core.indices import VectorStoreIndex, SimpleKeywordTableIndex, KeywordTableIndex
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.query_engine import (
    RouterQueryEngine,
    SubQuestionQueryEngine,
    TransformQueryEngine)
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.postprocessor.colbert_rerank import ColbertRerank
from llama_index.core.agent import ReActAgent



import groq
import kdbai_client as kdbai

import litellm
from litellm.integrations.opik.opik import OpikLogger
from opik import track
from opik.opik_context import get_current_span_data
from opik import Opik

import pandas as pd
from typing import List, Dict
import matplotlib.pyplot as plt
from llama_index.core.response.notebook_utils import display_response

import time
from tqdm.notebook import tqdm
from IPython.display import Markdown, display

import os
import nest_asyncio
nest_asyncio.apply()

# Configure Comet Opik

Traces can be observed on Comet's Opik after executing the next cell

https://docs.llamaindex.ai/en/stable/examples/observability/OpikCallback/

In [4]:
#Add the LiteLLM OpikTracker to log traces and steps to Opik:
os.environ["OPIK_PROJECT_NAME"] = "Omdena_SL_RAG_Agents_v3"
opik_logger = OpikLogger()
litellm.callbacks = [opik_logger]

In [5]:
# # Configure Opik from google secrets
from google.colab import userdata
os.environ["OPIK_API_KEY"]  = userdata.get('OPIK_API_KEY')
os.environ["OPIK_WORKSPACE"] = userdata.get('OPIK_WORKSPACE')

In [6]:
# setting Opik API Key
#os.environ["OPIK_API_KEY"] = (
#    os.environ["OPIK_API_KEY"]
#    if "OPIK_API_KEY" in os.environ
#    else getpass("OPIK API Key: ")
#)

#os.environ["OPIK_WORKSPACE"] = (
#    os.environ["OPIK_WORKSPACE"]
#    if "OPIK_WORKSPACE" in os.environ
#    else getpass("OPIK_WORKSPACE Key: ")
#)

In [7]:
set_global_handler("opik")

# Data Loading

In [8]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena_SL/data/v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [9]:
set([doc.metadata['class'] for doc in all_documents])

{'circular', 'guideline', 'regulatory'}

In [10]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [11]:
guideline_docs = [doc for doc in all_documents if doc.metadata['class'] == 'guideline']
len(guideline_docs)

13

In [12]:
regulatory_docs = [doc for doc in all_documents if doc.metadata['class'] == 'regulatory']
len(regulatory_docs)

47

# Create KDB.AI session and table¶

# Embedding Model

In [13]:
def setup_embedding_model():
    """
    Setup embedding model
    """
    model_name = 'BAAI/bge-small-en-v1.5'
    return FastEmbedEmbedding(
        model_name=model_name
        )

embed_model = setup_embedding_model()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

# Groq + KDBAI API Setup

In [14]:
from google.colab import userdata
GROQ_API_KEY = userdata.get('GROQ_API_KEY')
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')

In [15]:
def setup_groq_llm():
    """
    Setup Groq LLM
    """
    groq_api_key = GROQ_API_KEY
    if not groq_api_key:
        raise ValueError("Please set GROQ_API_KEY environment variable")

    return Groq(
        api_key=groq_api_key,
        model="llama-3.1-8b-instant",
        temperature=0.0
    )

def setup_google_llm():
    """
    Setup Gemini LLM
    """
    google_api_key = GOOGLE_API_KEY
    if not google_api_key:
        raise ValueError("Please set GOOGLE_API_KEY environment variable")

    return Gemini(
        api_key=google_api_key,
        model="models/gemini-1.5-flash",
        temperature=0.0
    )

llm = setup_google_llm()

# KDBAI API + Session Setup

In [16]:
KDBAI_API_KEY = userdata.get('KDBAI_API_KEY')
KDBAI_SESSION_ENDPOINT = userdata.get('KDBAI_ENDPOINT')

In [17]:
def setup_kdbai_api():
  """
  Setup KDBAI Session Endpoint and API
  """

  kdbai_endpoint = KDBAI_SESSION_ENDPOINT
  if not kdbai_endpoint:
        raise ValueError("Please set KDBAI_SESSION_ENDPOINT environment variable")

  kdbai_api_key = KDBAI_API_KEY
  if not kdbai_api_key:
        raise ValueError("Please set KDBAI_API_KEY environment variable")

  return kdbai.Session(
    endpoint=f"{kdbai_endpoint}",
    api_key=f"{kdbai_api_key}"
    )

session = setup_kdbai_api()

# KDBAI Vector Store Setup

## Session Database

In [18]:
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_rag_database_2",
 KDBAI database "srilanka_tea"]

In [19]:
# ensure no database called "srilanka_tea" exists
try:
    session.database("srilanka_rag_database").drop()
except kdbai.KDBAIException:
    pass

# Create the database
db = session.create_database("srilanka_rag_database")
session.databases()

[KDBAI database "default",
 KDBAI database "srilanka_rag_database",
 KDBAI database "srilanka_rag_database_2",
 KDBAI database "srilanka_tea"]

## Table Schema + Creation

In [20]:
# List all of the tables in the db
db.tables

[]

In [21]:
# Table - name & schema
table_name = "omdena_sl_agent"

# Same table schema and table index for each class
table_schema = [
        dict(name="document_id", type="bytes"),
        dict(name="text", type="bytes"),
        dict(name="embeddings", type="float32s"),
        dict(name="issue_date", type="str"),
        dict(name="issuing_authority", type="str"),
        dict(name="class", type="str"),
        dict(name="reference_number", type="str"),
        dict(name="title", type="str"),
        dict(name="original_doc_id", type="int16"),
    ]

indexFlat = {
        "name": "flat_index",
        "type": "flat",
        "column": "embeddings",
        "params": {'dims': 384, 'metric': 'CS'} # For similarity metric, choose from Euclidean Distance (L2), Dot Product (IP), or Cosine Similarity (CS).
    }

In [22]:
# First ensure the tables does not already exist
try:
    db.table(table_name).drop()
except kdbai.KDBAIException:
    pass

# Create tables
agent_table = db.create_table(table_name, table_schema, indexes=[indexFlat])

db.tables

[KDBAI table "omdena_sl_agent"]

In [23]:
agent_table.indexes

[{'name': 'flat_index',
  'type': 'flat',
  'column': 'embeddings',
  'params': {'metric': 'CS', 'dims': 384}}]

# Data Loading

In [24]:
def load_data(csv_path: str, text_col: List[str], metadata_cols: List[str]) -> List[Document]:
  """
  Load documents and include class in metadata
  """
  df = pd.read_csv(csv_path)
  df.fillna("nan", inplace=True)
  documents = []
  cols = ['original_doc_id', 'class', 'issuing_authority', 'title', 'issue_date', 'reference_number']
  for _, row in df.iterrows():
      text = str(row[text_col])
      doc = Document(
          text=text,
          metadata= {cols[i]: row[col] for i, col in enumerate(metadata_cols)}
      )
      documents.append(doc)
  return documents

DATA_PATH = "/content/drive/MyDrive/Omdena_SL/data/v0_LK_tea_dataset.csv"
text_col = 'markdown_content'
metadata_cols = ['id', 'class', 'issuing_authority', 'llama_title', 'llama_issue_date', 'llama_reference_number']

all_documents = load_data(DATA_PATH, text_col, metadata_cols)
len(all_documents)

167

In [25]:
set([doc.metadata['class'] for doc in all_documents])

{'circular', 'guideline', 'regulatory'}

In [26]:
circulars_docs = [doc for doc in all_documents if doc.metadata['class'] == 'circular']
len(circulars_docs)

107

In [27]:
guideline_docs = [doc for doc in all_documents if doc.metadata['class'] == 'guideline']
len(guideline_docs)

13

In [28]:
regulatory_docs = [doc for doc in all_documents if doc.metadata['class'] == 'regulatory']
len(regulatory_docs)

47

## Insert Data into Tables

In [29]:
Settings.llm = llm
Settings.embed_model = embed_model

In [30]:
splitter = SentenceSplitter(chunk_size=512, chunk_overlap=16)

nodes = splitter.get_nodes_from_documents(all_documents)

In [31]:
# uses SimpleVectorStore, SimpleDocumentStore, SimpleIndexStore to store embeddings, nodes and extra metadata respectively
storage_context = StorageContext.from_defaults()

storage_context.docstore.add_documents(nodes)

In [32]:
# uses regex to extract keywords
regex_keyword_index = SimpleKeywordTableIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)

# Summary Query Tool
summary_index = SummaryIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,)


# uses SimpleVectorStore to store embeddings
vector_index = VectorStoreIndex(
    nodes,
    storage_context=storage_context,
    show_progress=True,
)




Extracting keywords from nodes:   0%|          | 0/781 [00:00<?, ?it/s]

Generating embeddings:   0%|          | 0/781 [00:00<?, ?it/s]

OPIK: Started logging traces to the "Omdena_SL_RAG_Agents_v3" project at https://www.comet.com/opik/sherilyn-396/redirect/projects?name=Omdena_SL_RAG_Agents_v3.


In [33]:
from llama_index.core import PromptTemplate

# creating a custom prompt
QA_PROMPT_TMPL = """
Context:
---------------------
{context_str}
---------------------
Based on the context above, answer the question below. If the answer is not in the context,
inform the user without making up an answer. Additionally, provide a relevance score for the answer.

Question: {query_str}
Answer (with relevance score):
"""
QA_PROMPT = PromptTemplate(QA_PROMPT_TMPL)

#Uses Colbert V2 model as a reranker to rerank documents according to the fine-grained similarity between query tokens and passage tokens. Returns the top N ranked nodes.
colbert_reranker = ColbertRerank(
    top_n=5,
    model="colbert-ir/colbertv2.0",
    tokenizer="colbert-ir/colbertv2.0",
    keep_retrieval_score=True,
)

# Uses a cross encoder model as a reranker to rerank documents. Returns the top N ranked nodes.
crossencoder_reranker = SentenceTransformerRerank(
    model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=3
)


# regex keyword
keyword_query_engine = regex_keyword_index.as_query_engine(
    text_qa_template=QA_PROMPT,
    #node_postprocessors=[colbert_reranker]
)


# summary
summary_query_engine = summary_index.as_query_engine(
        response_mode="tree_summarize",
        use_async=True,
        #node_postprocessors=[colbert_reranker]
    )

# only vector semantic
vector_query_engine = vector_index.as_query_engine(text_qa_template=QA_PROMPT,
                                                   #node_postprocessors=[colbert_reranker]
                                                   )

# Vector and bm25
## When querying with hybrid mode, we can set similarity_top_k and sparse_top_k separately.
##sparse_top_k represents how many nodes will be retrieved from each dense and sparse query.
##For example, if sparse_top_k=5 is set, that means I will retrieve 5 nodes using sparse vectors and 5 nodes using dense vectors.
##similarity_top_k controls the final number of returned nodes.
##In the above setting, we end up with 10 nodes.
## A fusion algorithm is applied to rank and order the nodes from different vector spaces (relative score fusion in this case).
## similarity_top_k=2 means the top two nodes after fusion are returned.

hybrid_query_engine = vector_index.as_query_engine(
    vector_store_query_mode="hybrid", similarity_top_k=2, sparse_top_k=10, text_qa_template=QA_PROMPT
)



# HYDE
hyde = HyDEQueryTransform(
    include_original=True,
    )

hyde_query_engine = TransformQueryEngine(
    query_engine = vector_index.as_query_engine(),
    query_transform = hyde
    )



In [34]:
response = vector_query_engine.query(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)

In [35]:
print(response)

Answer: *Exobasidium vexans* Massee is a fungus that causes blister blight disease in tea plants. It spreads via wind-borne spores.  High relative humidity (85%) is required for spore production and infection; persistent mist and dew are particularly conducive to high infection levels.  Spore germination and release are favored by high moisture on the leaf surface, while direct sunlight or desiccation kill the spores.

Relevance Score: 1.0



In [36]:
response = keyword_query_engine.query(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)

In [37]:
print(response)

This question cannot be answered from the given source.  There is no mention of *Exobasidium vexans* or its spread in tea plantations in the provided text.

Relevance Score: 0



### QueryEngineTool

In [38]:
from llama_index.core.tools import QueryEngineTool


# keyword tool
keyword_tool = QueryEngineTool.from_defaults(
    query_engine=keyword_query_engine,
    name="keyword_engine",
    description="Useful when the question is not fully formed and consistes of keywords or topics.",
)


# summarization and listing tool
summary_tool = QueryEngineTool.from_defaults(
    query_engine=summary_query_engine,
    name="summary_engine",
    description="Useful when the user asks to summarize or list down points about a topic",
)


# semantic vector tool
vector_tool = QueryEngineTool.from_defaults(
    query_engine=vector_query_engine,
    name="vector_engine",
    description="Useful when the user asks specific context type fully-formed question.",
)

# vector and bm25
hybrid_tool = QueryEngineTool.from_defaults(
    query_engine=hybrid_query_engine,
    name="hybrid_engine",
    description="Useful when the user asks a fully-formed question.",
)


#Hyde
hyde_tool = QueryEngineTool(
    query_engine=hyde_query_engine,
    metadata=ToolMetadata(
        name="hyde_engine",
        description="Useful for answering complex queries",
    ),
)

# subquery
# Initialize SubQuestionQueryEngine
sub_question_query_engine = SubQuestionQueryEngine.from_defaults(
    query_engine_tools=[keyword_tool, summary_tool, vector_tool]
)

# Wrap SubQuestionQueryEngine in a QueryEngineTool
sub_question_tool = QueryEngineTool(
    query_engine=sub_question_query_engine,
    metadata=ToolMetadata(
        name="sub_question_engine",
        description="Useful when multiple queries are present in a question so breaking them into sub-questions",
    ),
)




In [39]:

from llama_index.core.response_synthesizers import TreeSummarize

TREE_SUMMARIZE_PROMPT_TMPL = """
Context from multiple sources is below. Each source may have a relevance score.

---------------------
{context_str}
---------------------

Based on the information from the sources above, answer the question below.

If the answer is not in the context, inform the user without making up an answer.

Question: {query_str}
Answer:
"""

tree_summarize = TreeSummarize(
    summary_template=PromptTemplate(TREE_SUMMARIZE_PROMPT_TMPL)
)


agent = ReActAgent.from_tools(
    tools = [vector_tool, sub_question_tool, summary_tool, keyword_tool],
    llm=llm,
    verbose=True,
    # context=context
)

In [40]:
response = agent.chat(
    "What is Exobasidium vexans and how does it spread in tea plantations?"
)
print(response)

> Running step d7f3bc17-e924-4351-8f4c-389baef225cb. Step input: What is Exobasidium vexans and how does it spread in tea plantations?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_engine
Action Input: {'input': 'Exobasidium vexans and its spread in tea plantations'}
Observation: Exobasidium vexans is a fungus that causes Blister Blight disease in tea plants. It is spread by wind-borne spores and requires high relative humidity (85%) for spore production and infection. The spores are killed when exposed to direct sunlight or desiccation. The disease is of great economic importance during rainy seasons at mid and high elevations.

Relevance score: 0.92

Note: The information is directly taken from the context, specifically from the section "# 1. Introduction" of the first advisory circular. The relevance score is high because the question is directly related to the content of the circular.
> Running step bdd086

In [41]:
display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

**`Final Response:`** Exobasidium vexans is a fungus that causes Blister Blight disease in tea plants. It spreads through wind-borne spores and requires high relative humidity of 85% for spore production and infection. The spores are sensitive to direct sunlight and desiccation, which can kill them. The disease is particularly significant during rainy seasons at mid and high elevations, making it a concern for tea plantations in these areas.

---

**`Source Node 1/2`**

**Node ID:** 10716f58-c1bf-49f5-b1d6-7f695529a1f7<br>**Similarity:** 0.7052035927772522<br>**Text:** # ADVISORY CIRCULAR

# No.DM JHL 925VynvT

Issued in: February 2024

Serial No. 04/24

# PROTECTION OF TEA FROM BLISTER BLIGHT

This Advisory Circular supersedes the Advisory Circular DM 1, Serial No. 1/02 issued in October 2002 and related previous Advisory Circulars and links with Advisory Circular PU 2.

# 1. Introduction

Blister blight disease of tea is caused by a fungus (*Exobasidium vexans* Massee) and is spread by wind-borne spores. High relative humidity (85%) is required for spore ...<br>**Metadata:** {'original_doc_id': 33, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'Protection of Tea from Blister Blight', 'issue_date': 'February 2024', 'reference_number': 'No. DM JHL 925VynvT, Serial No. 04/24', 'retrieval_score': 0.7470852703450455}<br>

---

**`Source Node 2/2`**

**Node ID:** 2c17775d-6695-4f1d-9006-a8a10993cebd<br>**Similarity:** 0.5475032329559326<br>**Text:** # ADVISORY CIRCULAR No.PM 5

# Issued in: February 2024

# Serial No. 15/24

# SAMPLING FOR NEMATODE ANALYSIS

This Advisory Circular supersedes the Advisory Circular PM 5 Serial No: 03/10 issued in December 2010 and related previous Advisory Circulars.

# 1. Introduction

All species of pathogenic nematodes of tea inhabit roots and soil. However, their number and activity will depend on soil factors such as soil temperature, moisture, depth, use of agro-chemicals and organic matter etc. and ...<br>**Metadata:** {'original_doc_id': 52, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'SAMPLING FOR NEMATODE ANALYSIS', 'issue_date': 'February 2024', 'reference_number': 'No.PM 5', 'retrieval_score': 0.7387041363987753}<br>

In [42]:
response.source_nodes

[NodeWithScore(node=TextNode(id_='10716f58-c1bf-49f5-b1d6-7f695529a1f7', embedding=None, metadata={'original_doc_id': 33, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'Protection of Tea from Blister Blight', 'issue_date': 'February 2024', 'reference_number': 'No. DM JHL 925VynvT, Serial No. 04/24', 'retrieval_score': 0.7470852703450455}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='66323fe1-80b6-4703-9af3-d0ff0f2c0f00', node_type='4', metadata={'original_doc_id': 33, 'class': 'circular', 'issuing_authority': 'Tea Research Institute', 'title': 'Protection of Tea from Blister Blight', 'issue_date': 'February 2024', 'reference_number': 'No. DM JHL 925VynvT, Serial No. 04/24'}, hash='29625e2f16d708c1d2fbfeb2c0ddfb7c889b0c05012e93341a723b3502b5739f'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='0ea8770a-462f-4274-b1f2-eff1c6a77425', node_type='1', metadata={}

In [43]:
agent.reset()
response = agent.chat(
    "What is the annual registration fee for the Tea factory?"
)

display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

> Running step c78f659b-2469-4f49-afce-1649b2baf3d8. Step input: What is the annual registration fee for the Tea factory?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: vector_engine
Action Input: {'input': 'Tea factory annual registration fee'}
Observation: The provided context does not mention the "Tea factory annual registration fee" explicitly. However, it does mention fees related to the registration and renewal of various entities involved in the tea industry, such as exporters, importers, packers, and warehouses.

Relevance score: 0.2

Note: The answer is not directly available in the context, and the relevance score is low, indicating that the context does not provide a clear answer to the question.
> Running step 9e886e21-a5c1-450a-8416-54e279f648bd. Step input: None
Thought: The vector engine was unable to provide a clear answer to the question, so I'll try using the keyword engine to see if it can provide m

**`Final Response:`** The annual registration fee for a tea factory varies based on its production capacity, with fees ranging from Rs. 25,000 for new applicants in the Large Category to Rs. 500,000 for renewal in the same category. However, the fees for new applicants in the Medium, Small, and Extra Small categories are not specified. For renewals, the fees are Rs. 125,000 for Medium Category, Rs. 50,000 for Small Category, and Rs. 25,000 for Extra Small Category.

---

**`Source Node 1/12`**

**Node ID:** 103dcbd3-810e-4236-82f7-8a570dd8703a<br>**Similarity:** 0.7245453596115112<br>**Text:** 500/-|
|Annual Statistical Bulletin|~do-|1,000/-|2,500/-|
|Monthly Statistical Summary|~do-|~No-|500/-|
|Statistical Reports|~do-|70/= per page|150/- per page|
|Annual Report|~do-|~No-|750/-|
|Issue of Quality Certificate|~do-|250/-|2,500/-|
|Tasting of tea sample by the Panel|~do-|~No-|500/- per sample|
|Lion Logo Registration|~do-|REG -4,500/- for 3 years|Registration-5,000/- per annum Renewal-5,000/-|
|Ozone Friendly Logo Registration|~do-|No|Registration-5,000/- Renewal-5,000/-|
|GI Regis...<br>**Metadata:** {'original_doc_id': 164, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': "LEVY OF FEES & CHARGES FOR BOARD'S SERVICES", 'issue_date': '01st September 2010', 'reference_number': 'OR/1/87', 'retrieval_score': 0.7564044338950707}<br>

...

**`Source Node 11/12`**

**Node ID:** 210b48bc-3201-4e8f-a227-483723c9b92f<br>**Similarity:** None<br>**Text:** 4. Where a registered manufacturer dies, the person, or any one of the persons, entering into possession or having control of the tea factory operated by that manufacturer, or having charge of the business of the deceased, shall give written information of the death to the Controller within one month after the date of the death, and shall be responsible for furnishing all such information in regard to that tea factory as may be required by the Controller, until the successor or each successor...<br>**Metadata:** {'original_doc_id': 19, 'class': 'regulatory', 'issuing_authority': 'Parliament of Sri Lanka', 'title': 'Tea Control Act, No. 51 of 1957', 'issue_date': 'nan', 'reference_number': '51 of 1957'}<br>

---

**`Source Node 12/12`**

**Node ID:** fabd2efd-0c75-4b28-96f0-834899a52bad<br>**Similarity:** None<br>**Text:** # TRANSLATIun

# Circuler Letter No-+iF / BJ, 62

Sri Lanka- Toa Board Division Sr

Tea Commissioner

P.0.Box 363, Colombo_1

30th August, 1984.

# To

All Registered Tea Manufacturers

# Dear Sir/Madam

# Payment of Reasonable Price for Green Leaf Tea Factories purchased by

Reference is requested to my Circular Letter No.iI/BL.60 of 6th February 1984, regarding the formula for calculating the reasonable price payable for green leaf.

The Government has decided to adopt the following measure...<br>**Metadata:** {'original_doc_id': 124, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Payment of Reasonable Price for Green Leaf Tea Factories purchased by', 'issue_date': '30th August, 1984', 'reference_number': 'Circular Letter No.II/BL.60 of 6th February 1984, and also Circular Letter No.I If / Bl. 62'}<br>

In [40]:
agent.reset()
response = agent.chat(
    "Summarize the tea control act of 1957"
)

display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

> Running step 64d33d66-acf1-4872-b6f2-532ad29bd86b. Step input: Summarize the tea control act of 1957
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: summary_engine
Action Input: {'input': 'Summarize the Tea Control Act of 1957'}
Observation: The Tea Control Act of 1957 outlines regulations for tea plantations and proprietors, tea manufacturers and factories, and the planting and replanting of tea.  It controls the possession, sale, and purchase of tea, as well as its export, along with tea seeds and other propagative plant parts.  The Act also establishes a Tea Advisory Board and a Tea Control Fund, and imposes an export duty on tea.  It details appeals processes and offenses, including penalties for non-compliance.  Specific regulations cover licensed dealers, declarations for tea delivery, restrictions on refuse tea, export licensing, and the administration of the Tea Control Fund.  The Act defines key terms such a

**`Final Response:`** The Tea Control Act of 1957 is a comprehensive piece of legislation regulating all aspects of the tea industry, from plantation management and manufacturing to the sale, export, and taxation of tea.  It established a Tea Advisory Board and a Tea Control Fund, and outlined penalties for non-compliance with its regulations.  The Act covers licensing, declarations, export controls, and the definition of key terms related to tea production and trade.

---

**`Source Node 1/781`**

**Node ID:** 42b3ce49-35bc-4961-a93c-036f0954787c<br>**Similarity:** None<br>**Text:** # Tea Subsidy

Acts Nos. 12 of 1958, 66 of 1961. 33 of 1966.

# AN ACT TO PROVIDE FOR THE ESTABLISHMENT OF A FUND FOR THE PURPOSE OF SUBSIDIZING THE REPLANTING AND REHABILITATION OF TEA ESTATES AND TEA SMALL HOLDINGS, THE REPLANTING OF RUBBER ESTATES AND RUBBER SMALL HOLDINGS WITH TEA, THE MARKETING OF GREEN TEA LEAF OF TEA ESTATES AND TEA SMALL HOLDINGS AND THE MANUFACTURE OF MADE TEA FROM SUCH GREEN TEA LEAF; AND FOR THE IMPOSITION OF AN EXPORT DUTY ON TEA TO BE PAID INTO SUCH FUND.

# Shor...<br>**Metadata:** {'original_doc_id': 0, 'class': 'regulatory', 'issuing_authority': 'Parliament of Sri Lanka', 'title': 'Tea Subsidy Act, No. 12 of 1958', 'issue_date': '19th September, 1958', 'reference_number': '12 of 1958, 66 of 1961, 33 of 1966'}<br>

...

**`Source Node 780/781`**

**Node ID:** b130d4da-0e99-4082-bdbb-ce51aa9370d9<br>**Similarity:** None<br>**Text:** not exceeding 1kg|Instant Tea Other Origin|
| | |2101.20.93|Immediate packing of content exceeding 1kg not exceeding 3kg|Instant Tea Other Origin|
| | |2101.20.99|3kg Instant Tea Other Origin|Instant Tea Other|
|2202|Tea in Liquid Beverage Form|2202.90.10|Tea in Liquid/Beverage Form Ready to Drink (RTD) Sri Lanka|RTD Packs|
| | |2202.90.90|Tea in Liquid/Beverage Form Origin|Ready to Drink (RTD) Other|

Below; When you prepare CUSDEC for exportation of tea kindly follow the guidelines appended...<br>**Metadata:** {'original_doc_id': 169, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'EXPORTATION OF TEA PREPARATION OF CUSTOMS GOODS DECLARATION (CUSDEC) FORM 53', 'issue_date': '02nd July 2010', 'reference_number': 'Circular No TCIEXIHS'}<br>

---

**`Source Node 781/781`**

**Node ID:** 1d05d91d-0a03-4d52-84aa-eab2df466b7f<br>**Similarity:** None<br>**Text:** Indicate Blend Sheet No or Nos: relating to the CUSDEC

# Cage (Box) 49 of CUSDEC Ler IL - Identification of Warehouse & Period

Indicate Tea Exporter Registration Number and Warehouse Registration Number of which the consignment is packed and ready for export:
of

55-

It

a

rets 05. Any further clarification on the above kindly contact the undersigned (Tel. 2508991/ Section (Tel. 2504090, Fax. 2584844, e-mail exports@pureceylontea.com) or Deputy Tea Commissioner, Tea Exports

els

k Yours ...<br>**Metadata:** {'original_doc_id': 169, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'EXPORTATION OF TEA PREPARATION OF CUSTOMS GOODS DECLARATION (CUSDEC) FORM 53', 'issue_date': '02nd July 2010', 'reference_number': 'Circular No TCIEXIHS'}<br>

In [41]:
agent.reset()
response = agent.chat(
    "green leaf dealer"
)

display_response(
    response, show_source=True, source_length=500, show_source_metadata=True
)

> Running step 66d114fc-d007-4d03-98bf-a2d38d0adfc4. Step input: green leaf dealer
Thought: The current language of the user is: English.  The input "green leaf dealer" is ambiguous and requires clarification. I will use the keyword_engine to attempt to expand on the keywords.
Action: keyword_engine
Action Input: {'input': 'green leaf dealer'}
Observation: The provided text frequently mentions "licensed dealers in green tea leaf".  These individuals are involved in the buying and selling of green tea leaf, subject to regulations outlined in the Tea Control Act and subsequent circulars.  They are required to maintain records, use scales, and follow specific procedures for payment and delivery.

Relevance Score: 10

> Running step 9235e79e-83e4-41e0-9a7e-156efce670ee. Step input: None
Thought: I can now answer without using any more tools. I'll use the user's language to answer.
Answer: Based on the keyword analysis, "green leaf dealer" likely refers to a licensed dealer of green tea lea

**`Final Response:`** Based on the keyword analysis, "green leaf dealer" likely refers to a licensed dealer of green tea leaves, subject to regulations governing the trade of tea.  These regulations likely include record-keeping, standardized weighing procedures, and specific payment and delivery protocols.

---

**`Source Node 1/10`**

**Node ID:** f2dba9ac-8db8-45a0-874e-71a239756869<br>**Similarity:** None<br>**Text:** # TRANSLATIun

# Circuler Letter No-+iF / BJ, 62

Sri Lanka- Toa Board Division Sr

Tea Commissioner

P.0.Box 363, Colombo_1

30th August, 1984.

# To

All Registered Tea Manufacturers

# Dear Sir/Madam

# Payment of Reasonable Price for Green Leaf Tea Factories purchased by

Reference is requested to my Circular Letter No.iI/BL.60 of 6th February 1984, regarding the formula for calculating the reasonable price payable for green leaf.

The Government has decided to adopt the following measure...<br>**Metadata:** {'original_doc_id': 124, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Payment of Reasonable Price for Green Leaf Tea Factories purchased by', 'issue_date': '30th August, 1984', 'reference_number': 'Circular Letter No.II/BL.60 of 6th February 1984, and also Circular Letter No.I If / Bl. 62'}<br>

...

**`Source Node 9/10`**

**Node ID:** 2a431200-6916-4f6d-8956-ec55ee539124<br>**Similarity:** None<br>**Text:** # Sri Lanka Tea Board

# Commissioner's Division

P.O. Box 363, Colombo

# Circular Letter No. ILEL66

2nd October, 1985

# TO: ALL REGISTERED

Dear Sir,

# Payment of Reasonable Price for Green Leaf Purchased by Tea Factories

Reference to requested paragraph 3 of my circular letter No. MF{BL,64 of 17th June 1985 regarding the formula for calculation the reasonable price payable for green leaf.

The Sri Lanka Tea Board has not decided to revise the present outturn percentage 22.22 (i.e. 4.5 ...<br>**Metadata:** {'original_doc_id': 125, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Payment of Reasonable Price for Green Leaf Purchased by Tea Factories', 'issue_date': '2nd October, 1985', 'reference_number': 'ILEL66'}<br>

---

**`Source Node 10/10`**

**Node ID:** fa5bfe1c-990a-4dce-80cb-f485506f1e03<br>**Similarity:** None<br>**Text:** # Circular No. TCIR/2004/1

Sri Lanka Tea Board

Tea Commissioner's Division

574, Galle Road,

Colombo 3.

21st October 2004.

# TO:

ALL REGISTERED TEA MANUFACTURERS & LICENSED DEALERS IN GREEN TEA LEAF

Dear Sir/Madam,

# COST OF TRANSPORT OF GREEN TEA LEAF

1. I refer to my Circular Nos. MIBL 151 of 29th June 2001 and MFMBL 152 of 10 August 2004.
2. You are hereby informed that the amounts allowed to be deducted as transport charges for green leaf by Registered Tea Manufacturers/Licensed ...<br>**Metadata:** {'original_doc_id': 156, 'class': 'circular', 'issuing_authority': 'Tea Board', 'title': 'Circular No. TCIR/2004/1', 'issue_date': '21st October 2004', 'reference_number': 'TCIR/2004/1'}<br>

# Using Edgar's RagEvaluator
https://github.com/Edgar454/rag_evaluator?tab=readme-ov-file

Needs a GLHF key. Can be obtained from https://glhf.chat/

In [44]:
%%capture
!pip install -q git+https://github.com/edgar454/rag_evaluator.git

In [45]:
# # Configure GLHF from google secrets
from google.colab import userdata
os.environ["GLHF_API_KEY"]  = userdata.get('GLHF_API_KEY')
os.environ["GLHF_URL"] = userdata.get('GLHF_URL')

In [ ]:
# setting Opik API Key
#os.environ["GLHF_API_KEY"] = (
#    os.environ["GLHF_API_KEY"]
#    if "GLHF_API_KEY" in os.environ
#    else getpass("GLHF_API_KEY: ")
#)

#os.environ["GLHF_URL"] = (
#    os.environ["GLHF_URL"]
#    if "GLHF_URL" in os.environ
#    else getpass("GLHF_URL Key: ")
#)

In [46]:
from rag_evaluator import RAGEvaluator
rageval = RAGEvaluator(query_engine=agent, api_key=os.environ["GLHF_API_KEY"] , mode = 'agentic')

Initializing the evaluator


  0%|          | 0/52 [00:00<?, ?it/s]

> Running step 8ac307cd-512b-4916-9219-f7fa0be31fa8. Step input: What are the main policies taken by SLTB to regulate the Refused Tea industry.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.  The question is specific enough to use the vector engine, but it also requires background knowledge about the SLTB and the "Refused Tea" industry, which I don't have.  I'll try the keyword engine first to see if I can get more context.
Action: keyword_engine
Action Input: {'input': 'SLTB Refused Tea industry regulation policies'}
Observation: This question cannot be answered from the given source.  There is no mention of the SLTB (Sri Lanka Tea Board) refusing any tea industry regulation policies.

Relevance Score: 0

> Running step 0ebb95ac-c3e3-4310-aa4a-40b4e6ac573a. Step input: None


  2%|▏         | 1/52 [00:07<06:05,  7.17s/it]

Thought: I cannot answer the question with the provided tools. The keyword engine search yielded no relevant information.  It's possible that "Refused Tea" is a misnomer or the SLTB is not involved in regulating it in the way the question implies.  More information is needed.
Answer: I am unable to answer your question.  My search for information on SLTB policies regarding a "Refused Tea" industry yielded no results.  The term "Refused Tea" may be unclear or inaccurate, or the SLTB may not be involved in its regulation.  Please provide more details or clarify your terminology.
> Running step d4c802f9-6908-425e-a6f6-2e8ee6a064e5. Step input: What are the minimum requirements I need to satisfy to be a Tea Factory Owner ?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: keyword_engine
Action Input: {'input': 'minimum requirements tea factory owner'}
Observation: The provided text mentions several requirements for tea facto

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Initialization complete


In [47]:
results = rageval.evaluate_rag()
print(results)

Beginning the evaluation
Evaluating Correctness


 54%|█████▍    | 28/52 [00:17<00:14,  1.64it/s]


Evaluating Faithfulness
Error generating completion : <html>
<head><title>413 Request Entity Too Large</title></head>
<body>
<center><h1>413 Request Entity Too Large</h1></center>
<hr><center>nginx</center>
</body>
</html>
Error evaluating faithfulness: expected string or bytes-like object


  0%|          | 0/28 [00:00<?, ?it/s]

Error generating completion : <html>
<head><title>413 Request Entity Too Large</title></head>
<body>
<center><h1>413 Request Entity Too Large</h1></center>
<hr><center>nginx</center>
</body>
</html>
Error evaluating faithfulness: expected string or bytes-like object
Error generating completion : <html>
<head><title>413 Request Entity Too Large</title></head>
<body>
<center><h1>413 Request Entity Too Large</h1></center>
<hr><center>nginx</center>
</body>
</html>
Error evaluating faithfulness: expected string or bytes-like object
Error generating completion : Connection error.
Error evaluating faithfulness: expected string or bytes-like object


...

 52%|█████▏    | 27/52 [14:22<09:48, 23.55s/it]

> Running step f90f5d97-9139-4f03-b4bb-efceb953b7de. Step input: What are the most suitable tree species for creating shelter belts in tea plantations across various regions?
Thought: The current language of the user is: English. I need to use a tool to help me answer the question about suitable tree species for shelter belts in tea plantations.  The question is specific enough to use the `vector_engine` directly.
Action: vector_engine
Action Input: {'input': 'What are the most suitable tree species for creating shelter belts in tea plantations across various regions?'}
Observation: The most suitable tree species for creating shelter belts in tea plantations vary by region:

* **Up-country:** A mixed stand of *Grevillea robusta*, and *Hakea saligna* or *Acacia* spp.
* **Mid-country:** A mixed stand of *Grevillea robusta* and *Acacia* spp.
* **Low-country:** A mixed stand of *Fragrea fragrans* and *Gliricidia sepium*.

Relevance Score: 1.0

> Running step 29cdd89e-8f01-4866-92e8-79f542a

Observation: Error: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
> Running step fa9b7fd2-eef6-4ea8-8a50-8533af18fc65. Step input: None


 54%|█████▍    | 28/52 [14:38<12:32, 31.37s/it]

Error answering question: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).
Error evaluating hallucinations: 'NoneType' object has no attribute 'response'
Evaluation complete
{'correctness_score': 3.4285714285714284, 'faithfulness_score': 7.608695652173913, 'retrieval_score': 10.0, 'hallucination_score': 4.844648035509246, 'total_score': 6.470478779063646}


In [ ]:
#table.drop()